In [1]:
# get train files
import glob

train_folders = ['CAR', 'UAH', 'UJITI', 'UJIUB']

files_names = []
n_files = 0
for building in train_folders:
    cur_file_names = glob.glob('train/' + building + '/log*')
    files_names.append(sorted(cur_file_names))
    n_files += len(cur_file_names)

print(files_names)
print('There are %d files' % n_files)

test_name = [files_names[1][2], files_names[1][3]]

[['train/CAR/logfile_CAR_R1_S3.txt', 'train/CAR/logfile_CAR_R1_S3mini.txt', 'train/CAR/logfile_CAR_R2_S3.txt', 'train/CAR/logfile_CAR_R2_S4.txt'], ['train/UAH/logfile_UAH_R1_S3.txt', 'train/UAH/logfile_UAH_R1_S4.txt', 'train/UAH/logfile_UAH_R2_S3.txt', 'train/UAH/logfile_UAH_R2_S4.txt', 'train/UAH/logfile_UAH_R4_S3.txt', 'train/UAH/logfile_UAH_R4_S4.txt'], ['train/UJITI/logfile_UJITI_R1_NEXUS5.txt', 'train/UJITI/logfile_UJITI_R2_NEXUS5.txt'], ['train/UJIUB/logfile_UJIUB_R1n_S3.txt', 'train/UJIUB/logfile_UJIUB_R1r_S3.txt', 'train/UJIUB/logfile_UJIUB_R2n_S3.txt', 'train/UJIUB/logfile_UJIUB_R2r_S3.txt', 'train/UJIUB/logfile_UJIUB_R3_S3.txt']]
There are 17 files


In [2]:
# import files into dataframe
import numpy as np
import pandas as pd

print('train')
train_track= []
for building in files_names:
    train_track.append([])
    for track_name in building:
        if not track_name in test_name:
            print(track_name)
            train_track[-1].append(pd.read_csv(track_name, sep=';', engine='c', names=range(11)))
            print(np.unique(train_track[-1][-1][0]))

print('test')
test_track = []
for building in files_names:
    test_track.append([])
    for track_name in building:
        if track_name in test_name:
            print(track_name)
            test_track[-1].append(pd.read_csv(track_name, sep=';', engine='c', names=range(11)))
            print(np.unique(test_track[-1][-1][0]))


train
train/CAR/logfile_CAR_R1_S3.txt
['ACCE' 'AHRS' 'GNSS' 'GYRO' 'LIGH' 'MAGN' 'POSI' 'PRES' 'SOUN' 'WIFI']
train/CAR/logfile_CAR_R1_S3mini.txt
['ACCE' 'AHRS' 'GNSS' 'GYRO' 'MAGN' 'POSI' 'SOUN' 'WIFI']
train/CAR/logfile_CAR_R2_S3.txt
['ACCE' 'AHRS' 'GNSS' 'GYRO' 'LIGH' 'MAGN' 'POSI' 'PRES' 'PROX' 'SOUN'
 'WIFI']
train/CAR/logfile_CAR_R2_S4.txt
['ACCE' 'AHRS' 'GNSS' 'GYRO' 'HUMI' 'LIGH' 'MAGN' 'POSI' 'PRES' 'PROX'
 'SOUN' 'TEMP' 'WIFI']
train/UAH/logfile_UAH_R1_S3.txt
['ACCE' 'AHRS' 'GNSS' 'GYRO' 'LIGH' 'MAGN' 'POSI' 'PRES' 'SOUN' 'WIFI']
train/UAH/logfile_UAH_R1_S4.txt
['ACCE' 'AHRS' 'GNSS' 'GYRO' 'HUMI' 'LIGH' 'MAGN' 'POSI' 'PRES' 'SOUN'
 'TEMP' 'WIFI']
train/UAH/logfile_UAH_R4_S3.txt
['ACCE' 'AHRS' 'GNSS' 'GYRO' 'LIGH' 'MAGN' 'POSI' 'PRES' 'SOUN' 'WIFI']
train/UAH/logfile_UAH_R4_S4.txt
['ACCE' 'AHRS' 'GNSS' 'GYRO' 'HUMI' 'LIGH' 'MAGN' 'POSI' 'PRES' 'SOUN'
 'TEMP' 'WIFI']
train/UJITI/logfile_UJITI_R1_NEXUS5.txt
['ACCE' 'AHRS' 'GNSS' 'GYRO' 'LIGH' 'MAGN' 'POSI' 'PRES' 'PROX' 'SOUN'
 

/usr/local/lib/python3.4/dist-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# get position
def get_posi(track_df):
    posi_df = track_df.iloc[track_df[0].values == 'POSI']
    posi_df = posi_df.drop([0, 7, 8, 9 ,10], axis=1).astype(float)
    posi_df.columns = ['AppTime', 'SensorTime', 'Lat', 'Lon', 'FloorID', 'BuildingID']
    return posi_df

train_posi = []
for i in range(len(train_track)):
    train_posi.append([])
    for j in range(len(train_track[i])):
        train_posi[i].append(get_posi(train_track[i][j]))

test_posi = []
for i in range(len(test_track)):
    test_posi.append([])
    for j in range(len(test_track[i])):
        test_posi[i].append(get_posi(test_track[i][j]))
        
# Pad ground truth
def pad_position(pos_df, track_df):
    pos_pad_start = pos_df.iloc[0]
    pos_pad_start.at['AppTime'] = 0
    pos_pad_end = pos_df.iloc[-1]
    pos_pad_end.at['AppTime'] = track_df[1].values[-1]
    pos_array = np.vstack((pos_pad_start.values.reshape((1, pos_df.shape[1])), 
                            pos_df.values, 
                            pos_pad_end.values.reshape((1, pos_df.shape[1]))))
    pad_pos = pd.DataFrame(pos_array, columns=pos_df.columns.values)
    return pad_pos

for i in range(len(train_track)):
    for j in range(len(train_track[i])):
        train_posi[i][j] = pad_position(train_posi[i][j], train_track[i][j])

for i in range(len(test_track)):
    for j in range(len(test_track[i])):
        test_posi[i][j] = pad_position(test_posi[i][j], test_track[i][j])

In [4]:
# Associate buildingID and floorID to line
train_build = []
for i in range(len(train_track)):
    train_build.append([])
    for j in range(len(train_track[i])):
        train_build[i].append(train_posi[i][j]['BuildingID'].iloc[0])

test_build = []
for i in range(len(test_track)):
    test_build.append([])
    for j in range(len(test_track[i])):
        test_build[i].append(test_posi[i][j]['BuildingID'].iloc[0])
print(train_build)
print(test_build)

[[10.0, 10.0, 10.0, 10.0], [20.0, 20.0, 20.0, 20.0], [40.0, 40.0], [20.0, 20.0, 30.0, 30.0, 0.0]]
[[], [20.0, 20.0], [], []]


In [5]:
# Create building mac wifi database
unique_macs = {10: [], 20: [], 40: []}
for i in range(len(train_track)):
    for j in range(len(train_track[i])):
        buildid = train_build[i][j]
        wifi_macs = train_wifi[i][j]['MAC'].values
        unique_macs[buildid] = list(np.unique(wifi_macs)) + list(unique_macs[buildid])
    print(len(unique_macs[buildid]))

NameError: name 'train_wifi' is not defined

In [26]:
# create wifi database
def get_wifi(track_df):
    wifi_df = track_df.iloc[track_df[0].values == 'WIFI']
    wifi_df = wifi_df.drop([0, 3, 6, 7, 8, 9, 10], axis=1)
    wifi_df.columns = ['Apptime', 'Sensortime', 'MAC', 'dBm']
    return wifi_df

train_wifi = []
for i in range(len(train_track)):
    train_wifi.append([])
    for j in range(len(train_track[i])):
        train_wifi[i].append(get_wifi(train_track[i][j]))

test_wifi = []
for i in range(len(test_track)):
    test_wifi.append([])
    for j in range(len(test_track[i])):
        test_wifi[i].append(get_wifi(test_track[i][j]))

In [39]:
# calculat
def dummy_wifi(wifi_df, mac_building):
    wifi_times = np.unique(wifi_df['AppTime'].values) 
    for i in range(len(wifi_times)):
        cur_wifi = wifi_df.iloc[wifi_df['AppTime'].values == wifi_times[i]]
        for buildid in unique_macs:
            print(unique_macs[buildid])
    return 0


wifi_dummy_data = []
for i in range(len(wifi_data)):
    wifi_dummy_data.append([])
    for j in range(len(wifi_data[i])):
        wifi_samples_cur = np.unique(wifi_data[i][j]['AppTime'].values)
        wifi_dummy_cur = np.ones((wifi_samples_cur.shape[0], len(building_macs[i]))) * -200
        wifi_dummy_cur = pd.DataFrame(wifi_dummy_cur, columns=building_macs[i], index=wifi_samples_cur)
        print(i, j, wifi_samples_cur.shape[0])
        for k, sample_time in enumerate(wifi_samples_cur):
            wifi_cur = wifi_data[i][j].iloc[wifi_data[i][j]['AppTime'].values == sample_time]
            for l in range(wifi_cur.shape[0]):
                wifi_dummy_cur[wifi_cur['MAC'].iat[l]].iat[k] = wifi_cur['rssi'].iat[l]
        wifi_dummy_data[i].append(wifi_dummy_cur)
print(wifi_dummy_data[0][0].head())

40
['00:0c:42:62:15:cb', '00:12:17:de:2b:03', '00:13:80:43:d1:50', '00:13:80:43:d1:51', '00:13:c3:44:af:f1', '00:13:c3:44:b0:70', '00:13:c3:44:b0:71', '00:13:c3:44:b5:60', '00:13:c3:44:b5:61', '00:13:c3:44:c7:20', '00:13:c3:44:c7:21', '00:13:c3:44:d6:70', '00:13:c3:44:d6:71', '00:13:c3:44:d6:e0', '00:13:c3:44:d6:e1', '00:13:c3:44:d8:20', '00:13:c3:44:d8:21', '00:13:c3:44:d8:e0', '00:13:c3:44:d8:e1', '00:13:c3:44:d9:b0', '00:13:c3:44:d9:b1', '00:13:c3:44:ee:f0', '00:13:c3:44:ee:f1', '00:14:1c:dd:f1:e0', '00:14:1c:dd:f1:e1', '00:14:1c:dd:f3:40', '00:14:1c:dd:f3:41', '00:14:1c:dd:f6:70', '00:14:1c:dd:f6:71', '00:14:1c:dd:fa:a0', '00:14:1c:dd:fa:a1', '00:14:1c:dd:fc:40', '00:14:1c:dd:fc:41', '00:14:1c:de:93:a0', '00:14:1c:de:93:a1', '00:14:69:f2:31:40', '00:14:69:f2:31:41', '00:14:69:f2:35:50', '00:14:69:f2:35:51', '00:14:69:f2:39:c0', '00:14:69:f2:39:c1', '00:14:69:f2:3b:80', '00:14:69:f2:3b:81', '00:15:6d:a9:bb:8f', '00:15:6d:d8:17:96', '00:15:6d:de:da:15', '00:15:6d:e4:7d:15', '00:17:59

KeyError: 0